In [1]:
pip install redis

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [84]:
import redis
import requests

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)

# Download the dataset
url = "https://raw.githubusercontent.com/RediSearch/redisearch-getting-started/master/sample-app/redisearch-docker/dataset/import_movies.redis"
response = requests.get(url)
commands = response.text.split('\n')

for cmd in commands:
    if not cmd.strip() or cmd.startswith("#"):  # Skip empty/comments
        continue
    
    parts = cmd.strip().split()
    command = parts[0].upper()
    
    try:
        if command == "HSET":
            key = parts[1]
            # Convert remaining parts to a dictionary: {"field1": "val1", "field2": "val2"...}
            fields_values = parts[2:]
            mapping = dict(zip(fields_values[::2], fields_values[1::2]))
            r.hset(key, mapping=mapping)
        
        elif command == "FT.CREATE":
            # Handle RediSearch commands as raw strings
            r.execute_command(*parts)
        
        else:
            r.execute_command(*parts)
    
    except Exception as e:
        print(f"Failed: {cmd}\nError: {e}")

print("Import complete!")

Import complete!


In [139]:
results = []
for key in r.scan_iter(match="theater:*"):
    title = r.hget(key, "name")

print(title)

Ah-ga-ssi


In [127]:
# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

def find_theaters_by_city(city):
    theaters = []
    # Iterate all theater keys (uses SCAN for safety)
    for key in r.scan_iter("theater:*"):
        theater_data = r.hgetall(key)
        if theater_data.get(b'city', b'').decode('utf-8').lower() == city.lower():
            theaters.append({
                "id": key.decode('utf-8'),
                "name": theater_data.get(b'name', b'').decode('utf-8'),
                "city": theater_data.get(b'city', b'').decode('utf-8'),
                "state": theater_data.get(b'state', b'').decode('utf-8')
            })
    return theaters

# Find theaters in New York
ny_theaters = find_theaters_by_city("New York")

# Print results
print(f"Found {len(ny_theaters)} theaters in New York:")
for theater in ny_theaters:
    print(f"\nID: {theater['id']}")
    print(f"Name: {theater['name']}")
    print(f"Location: {theater['city']}, {theater['state']}")

Found 0 theaters in New York:


In [128]:
import redis
import requests

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

# Download the dataset
url = "https://raw.githubusercontent.com/RediSearch/redisearch-getting-started/master/sample-app/redisearch-docker/dataset/import_theaters.redis"
response = requests.get(url)
commands = response.text.split('\n')

for cmd in commands:
    if not cmd.strip() or cmd.startswith("#"):  # Skip empty/comments
        continue
    
    parts = cmd.strip().split()
    command = parts[0].upper()
    
    try:
        if command == "HSET":
            key = parts[1]
            # Convert remaining parts to a dictionary: {"field1": "val1", "field2": "val2"...}
            fields_values = parts[2:]
            mapping = dict(zip(fields_values[::2], fields_values[1::2]))
            r.hset(key, mapping=mapping)
        
        elif command == "FT.CREATE":
            # Handle RediSearch commands as raw strings
            r.execute_command(*parts)
        
        else:
            r.execute_command(*parts)
    
    except Exception as e:
        print(f"Failed: {cmd}\nError: {e}")

print("Import complete!")

Import complete!


In [129]:
import redis

r = redis.Redis()
results = []

for key in r.scan_iter(match="movie:*"):
    title = r.hget(key, "title").decode('utf-8')
    if "The Lord of the Rings" in title:
        results.append(title)

In [130]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

def find_theaters_by_city(city):
    theaters = []
    # Iterate all theater keys (uses SCAN for safety)
    for key in r.scan_iter("theater:*"):
        theater_data = r.hgetall(key)
        if theater_data.get(b'city', b'').decode('utf-8').lower() == city.lower():
            theaters.append({
                "id": key.decode('utf-8'),
                "name": theater_data.get(b'name', b'').decode('utf-8'),
                "city": theater_data.get(b'city', b'').decode('utf-8'),
                "state": theater_data.get(b'state', b'').decode('utf-8')
            })
    return theaters

# Find theaters in New York
ny_theaters = find_theaters_by_city("New York")

# Print results
print(f"Found {len(ny_theaters)} theaters in New York:")
for theater in ny_theaters:
    print(f"\nID: {theater['id']}")
    print(f"Name: {theater['name']}")
    print(f"Location: {theater['city']}, {theater['state']}")

Found 0 theaters in New York:


In [131]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

def get_top_rated_movies(limit=5):
    # Initialize a list to store (rating, movie_id, movie_data)
    movies_with_ratings = []
    
    # Iterate through all movie keys
    for key in r.scan_iter("movie:*"):
        movie_data = r.hgetall(key)
        rating = float(movie_data.get(b'rating', 0))  # Default to 0 if missing
        movies_with_ratings.append((rating, key, movie_data))
    
    # Sort by rating (descending) and take top 'limit' movies
    top_movies = sorted(movies_with_ratings, key=lambda x: x[0], reverse=True)[:limit]
    
    # Format results
    results = []
    for rating, key, movie_data in top_movies:
        results.append({
            "id": key.decode('utf-8'),
            "title": movie_data.get(b'title', b'N/A').decode('utf-8'),
            "year": movie_data.get(b'year', b'N/A').decode('utf-8'),
            "rating": rating,
            "genre": movie_data.get(b'genre', b'N/A').decode('utf-8')
        })
    return results

# Get and print top 5 movies
top_5_movies = get_top_rated_movies(limit=5)
print("Top 5 Movies by Rating:")
for idx, movie in enumerate(top_5_movies, start=1):
    print(f"\n{idx}. {movie['title']} ({movie['year']})")
    print(f"   Rating: {movie['rating']}")
    print(f"   Genre: {movie['genre']}")

Top 5 Movies by Rating:

1. Hacksaw Ridge (N/A)
   Rating: 8.8
   Genre: Biography

2. Star Wars: Episode V - The Empire Strikes Back (N/A)
   Rating: 8.7
   Genre: Action

3. Moonlight (N/A)
   Rating: 8.7
   Genre: Drama

4. La La Land (N/A)
   Rating: 8.6
   Genre: Comedy

5. Interstellar (N/A)
   Rating: 8.6
   Genre: Adventure


In [132]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

def get_movies_released_after(year_threshold=2010):
    movies = []
    # Iterate all movie keys safely with SCAN
    for key in r.scan_iter("movie:*"):
        movie_data = r.hgetall(key)
        year = int(movie_data.get(b'year', 0))  # Default to 0 if missing
        if year > year_threshold:
            movies.append({
                "id": key.decode('utf-8'),
                "title": movie_data.get(b'title', b'N/A').decode('utf-8'),
                "year": year,
                "rating": float(movie_data.get(b'rating', 0)),
                "genre": movie_data.get(b'genre', b'N/A').decode('utf-8')
            })
    # Sort by year (newest first)
    movies.sort(key=lambda x: x['year'], reverse=True)
    return movies

# Get movies released after 2010
recent_movies = get_movies_released_after(2010)

# Print results
print(f"Found {len(recent_movies)} movies released after 2010:")
for movie in recent_movies:
    print(f"\nTitle: {movie['title']}")
    print(f"Year: {movie['year']}")
    print(f"Rating: {movie['rating']}")
    print(f"Genre: {movie['genre']}")

Found 1 movies released after 2010:

Title: Dune: Part Two
Year: 2024
Rating: 7.0
Genre: Biography


In [133]:
import redis

r = redis.Redis(host='localhost', port=6379, db=0)

def get_movies_released_after(year_threshold=2010):
    movies = []
    for key in r.scan_iter("movie:*"):
        movie_data = r.hgetall(key)
        year_str = movie_data.get(b'year')
        if not year_str:
            continue  # Skip movies without a year
        try:
            year = int(year_str.decode())
            if year > year_threshold:
                movies.append({
                    "id": key.decode(),
                    "title": movie_data.get(b'title', b'N/A').decode(),
                    "year": year,
                    "genre": movie_data.get(b'genre', b'N/A').decode()
                })
        except ValueError:
            print(f"Invalid year for {key}: {year_str}")
    movies.sort(key=lambda x: x['year'], reverse=True)
    return movies

recent_movies = get_movies_released_after(2010)
print(f"Found {len(recent_movies)} movies after 2010:")
for movie in recent_movies:
    print(f"\nTitle: {movie['title']} ({movie['year']})")

Found 1 movies after 2010:

Title: Dune: Part Two (2024)


In [134]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

def get_movies_by_genre(target_genre="Action"):
    action_movies = []
    # Iterate all movie keys safely with SCAN
    for key in r.scan_iter("movie:*"):
        movie_data = r.hgetall(key)
        genre = movie_data.get(b'genre', b'').decode('utf-8').lower()
        if genre == target_genre.lower():
            action_movies.append({
                "id": key.decode('utf-8'),
                "title": movie_data.get(b'title', b'N/A').decode('utf-8'),
                "year": movie_data.get(b'year', b'N/A').decode('utf-8'),
                "genre": genre,
                "rating": movie_data.get(b'rating', b'N/A').decode('utf-8')
            })
    return action_movies

# Get all Action movies
action_movies = get_movies_by_genre("Action")

# Print results
print(f"Found {len(action_movies)} Action movies:")
for movie in action_movies:
    print(f"\nTitle: {movie['title']}")
    print(f"Year: {movie['year']}")
    print(f"Rating: {movie['rating']}")

Found 102 Action movies:

Title: The Expendables 3
Year: N/A
Rating: 6.1

Title: Doctor Strange
Year: N/A
Rating: 8.0

Title: Star Trek Beyond
Year: N/A
Rating: 7.2

Title: Captain America: The Winter Soldier
Year: N/A
Rating: 7.8

Title: Fury
Year: N/A
Rating: 7.6

Title: Grimsby
Year: N/A
Rating: 6.2

Title: Busanhaeng
Year: N/A
Rating: 7.6

Title: Vampire Academy
Year: N/A
Rating: 5.6

Title: Chappie
Year: N/A
Rating: 6.9

Title: The Equalizer
Year: N/A
Rating: 7.2

Title: Central Intelligence
Year: N/A
Rating: 6.4

Title: Free State of Jones
Year: N/A
Rating: 6.9

Title: Point Break
Year: N/A
Rating: 5.3

Title: Left Behind
Year: N/A
Rating: 3.1

Title: Guardians of the Galaxy Vol. 2
Year: N/A
Rating: 7.6

Title: Deepwater Horizon
Year: N/A
Rating: 7.5

Title: The Maze Runner
Year: N/A
Rating: 6.8

Title: Top Gun
Year: N/A
Rating: 6.9

Title: Batman v Superman: Dawn of Justice
Year: N/A
Rating: 6.8

Title: The Finest Hours
Year: N/A
Rating: 6.8

Title: Seventh Son
Year: N/A
Rating:

In [135]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

def get_movies_by_rating_range(min_rating=7.0, max_rating=9.0):
    matched_movies = []
    # Iterate all movie keys safely with SCAN
    for key in r.scan_iter("movie:*"):
        movie_data = r.hgetall(key)
        try:
            rating = float(movie_data.get(b'rating', 0))  # Default to 0 if missing
            if min_rating <= rating <= max_rating:
                matched_movies.append({
                    "id": key.decode('utf-8'),
                    "title": movie_data.get(b'title', b'N/A').decode('utf-8'),
                    "year": movie_data.get(b'year', b'N/A').decode('utf-8'),
                    "rating": rating,
                    "genre": movie_data.get(b'genre', b'N/A').decode('utf-8')
                })
        except ValueError:
            print(f"Invalid rating for {key}: {movie_data.get(b'rating')}")
    # Sort by rating (descending)
    matched_movies.sort(key=lambda x: x['rating'], reverse=True)
    return matched_movies

# Get movies with ratings between 7.0 and 9.0
filtered_movies = get_movies_by_rating_range(7.0, 9.0)

# Print results
print(f"Found {len(filtered_movies)} movies with ratings between 7.0 and 9.0:")
for movie in filtered_movies:
    print(f"\nTitle: {movie['title']} ({movie['year']})")
    print(f"Rating: {movie['rating']:.1f}")
    print(f"Genre: {movie['genre']}")

Found 134 movies with ratings between 7.0 and 9.0:

Title: Hacksaw Ridge (N/A)
Rating: 8.8
Genre: Biography

Title: Star Wars: Episode V - The Empire Strikes Back (N/A)
Rating: 8.7
Genre: Action

Title: Moonlight (N/A)
Rating: 8.7
Genre: Drama

Title: La La Land (N/A)
Rating: 8.6
Genre: Comedy

Title: Interstellar (N/A)
Rating: 8.6
Genre: Adventure

Title: Star Wars: Episode IV - A New Hope (N/A)
Rating: 8.6
Genre: Action

Title: Arrival (N/A)
Rating: 8.5
Genre: Drama

Title: Whiplash (N/A)
Rating: 8.5
Genre: Drama

Title: Vincent (N/A)
Rating: 8.4
Genre: Short

Title: Star Wars: Episode VI - Return of the Jedi (N/A)
Rating: 8.3
Genre: Action

Title: Lucifer (N/A)
Rating: 8.3
Genre: Crime

Title: Room (N/A)
Rating: 8.2
Genre: Drama

Title: Hell or High Water (N/A)
Rating: 8.2
Genre: Crime

Title: Inside Out (N/A)
Rating: 8.2
Genre: Animation

Title: Nocturnal Animals (N/A)
Rating: 8.1
Genre: Drama

Title: Relatos salvajes (N/A)
Rating: 8.1
Genre: Comedy

Title: The Revenant (N/A)
Ratin

In [136]:
#additional challenge 1
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

def get_top_rated_movies(limit=10):
    movies = []
    # Iterate all movie keys safely with SCAN
    for key in r.scan_iter("movie:*"):
        movie_data = r.hgetall(key)
        try:
            rating = float(movie_data.get(b'rating', 0))  # Default to 0 if missing
            movies.append({
                "id": key.decode('utf-8'),
                "title": movie_data.get(b'title', b'N/A').decode('utf-8'),
                "year": movie_data.get(b'year', b'N/A').decode('utf-8'),
                "rating": rating,
                "genre": movie_data.get(b'genre', b'N/A').decode('utf-8')
            })
        except ValueError:
            print(f"Invalid rating for {key}: {movie_data.get(b'rating')}")
    
    # Sort by rating (descending) and return top N
    top_movies = sorted(movies, key=lambda x: x['rating'], reverse=True)[:limit]
    return top_movies

# Get top 10 movies
top_10_movies = get_top_rated_movies(limit=10)

# Print results
print("Top 10 Highest-Rated Movies:")
for idx, movie in enumerate(top_10_movies, start=1):
    print(f"\n{idx}. {movie['title']} ({movie['year']})")
    print(f"   Rating: {movie['rating']:.1f}")
    print(f"   Genre: {movie['genre']}")

Top 10 Highest-Rated Movies:

1. Hacksaw Ridge (N/A)
   Rating: 8.8
   Genre: Biography

2. Star Wars: Episode V - The Empire Strikes Back (N/A)
   Rating: 8.7
   Genre: Action

3. Moonlight (N/A)
   Rating: 8.7
   Genre: Drama

4. La La Land (N/A)
   Rating: 8.6
   Genre: Comedy

5. Interstellar (N/A)
   Rating: 8.6
   Genre: Adventure

6. Star Wars: Episode IV - A New Hope (N/A)
   Rating: 8.6
   Genre: Action

7. Arrival (N/A)
   Rating: 8.5
   Genre: Drama

8. Whiplash (N/A)
   Rating: 8.5
   Genre: Drama

9. Vincent (N/A)
   Rating: 8.4
   Genre: Short

10. Star Wars: Episode VI - Return of the Jedi (N/A)
   Rating: 8.3
   Genre: Action


In [137]:
#Add'l Challenge 2
import time

# When adding a new movie:
r.hset(
    "movie:123",
    mapping={
        "title": "Dune: Part Two",
        "year": "2024",
        "created_at": int(time.time())  # Current Unix timestamp
    }
)

0

In [138]:
import redis

r = redis.Redis(host='localhost', port=6379, db=0)

def get_recent_movies(limit=3):
    movies = []
    for key in r.scan_iter("movie:*"):
        movie_data = r.hgetall(key)
        movies.append({
            "id": key.decode(),
            "title": movie_data.get(b'title', b'N/A').decode(),
            "year": movie_data.get(b'year', b'N/A').decode(),
            "created_at": int(movie_data.get(b'created_at', 0))
        })
    # Sort by timestamp (newest first)
    movies.sort(key=lambda x: x['created_at'], reverse=True)
    return movies[:limit]

recent_movies = get_recent_movies(limit=3)
print("3 Most Recently Added Movies:")
for movie in recent_movies:
    print(f"\nTitle: {movie['title']}")
    print(f"Year: {movie['year']}")
    print(f"Added on: {movie['created_at']}")  # Convert to datetime if needed

3 Most Recently Added Movies:

Title: Dune: Part Two
Year: 2024
Added on: 1754400404

Title: Hacksaw Ridge
Year: N/A
Added on: 0

Title: Miss Peregrine's Home for Peculiar Children
Year: N/A
Added on: 0
